<a href="https://colab.research.google.com/github/kvgopichand1/food_reciepe/blob/main/Food_receipe_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

class SimplePizzaGenerator:
    def __init__(self):
        self.tokenizer = Tokenizer()
        self.model = None
        self.max_len = 20  # Reduced sequence length for better learning

    def create_data(self):
        """Create basic pizza recipe dataset with more variety"""
        recipes = {
            'instructions': [
                "Mix flour water salt yeast to make dough. Let rise for 1 hour. Add toppings. Bake at 450F for 15 minutes.",
                "Combine flour water yeast salt sugar. Knead for 10 minutes. Rest for 2 hours. Add sauce cheese. Bake 475F 12 minutes.",
                "Make dough with flour water salt. Rise for 3 hours. Stretch dough. Add toppings. Bake at 500F for 10 minutes.",
                "Blend flour salt yeast water. Rest overnight. Shape dough. Top with ingredients. Bake at 425F for 20 minutes.",
                "Prepare dough: mix flour, water, salt, olive oil. Let rise for 1.5 hours. Add tomato sauce and mozzarella. Bake at 400F for 18 minutes.",
                "Create dough by combining flour, water, yeast, and salt. Proof for 2 hours. Add your favorite toppings. Bake at 475F for 14 minutes.",
                "Mix whole wheat flour, water, salt, and yeast. Rest for 1 hour. Top with vegetables. Bake at 450F for 16 minutes.",
                "Combine bread flour, water, salt, and sourdough starter. Ferment for 4 hours. Add toppings. Bake at 500F for 11 minutes."
            ] * 25  # Repeat to create more samples (200 total)
        }
        return pd.DataFrame(recipes)

    def prepare_data(self, df):
        """Tokenize and prepare training data"""
        texts = df['instructions'].values
        self.tokenizer.fit_on_texts(texts)
        sequences = self.tokenizer.texts_to_sequences(texts)

        vocab_size = len(self.tokenizer.word_index) + 1
        print(f"Vocabulary size: {vocab_size}")

        # Create training sequences
        input_sequences = []
        for sequence in sequences:
            for i in range(1, len(sequence)):
                n_gram_sequence = sequence[:i+1]
                input_sequences.append(n_gram_sequence)

        max_sequence_len = max([len(x) for x in input_sequences])
        padded_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

        X = padded_sequences[:, :-1]
        y = padded_sequences[:, -1]
        y = to_categorical(y, num_classes=vocab_size)

        return X, y, max_sequence_len

    def build_model(self, vocab_size, max_sequence_len):
        """Improved LSTM model for recipe generation"""
        model = Sequential([
            Embedding(vocab_size, 64, input_length=max_sequence_len-1),
            LSTM(128, return_sequences=True),
            Dropout(0.2),
            LSTM(64),
            Dense(64, activation='relu'),
            Dense(vocab_size, activation='softmax')
        ])
        model.compile(loss='categorical_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])
        return model

    def train(self, X, y, max_sequence_len, epochs=30):
        """Train the model with early stopping"""
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        vocab_size = y.shape[1]
        self.model = self.build_model(vocab_size, max_sequence_len)

        early_stop = EarlyStopping(monitor='val_loss', patience=5)

        history = self.model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=epochs,
            batch_size=64,
            callbacks=[early_stop]
        )
        return history

    def generate_recipe(self, start_text, num_words=50):
        """Generate a new recipe from seed text"""
        if not self.model:
            raise Exception("Model not trained yet. Call train() first.")

        generated_text = start_text
        for _ in range(num_words):
            token_list = self.tokenizer.texts_to_sequences([generated_text])[0]
            token_list = pad_sequences([token_list], maxlen=self.max_len-1, padding='pre')

            predicted_probs = self.model.predict(token_list, verbose=0)
            predicted = np.argmax(predicted_probs, axis=-1)[0]

            output_word = ""
            for word, index in self.tokenizer.word_index.items():
                if index == predicted:
                    output_word = word
                    break

            generated_text += " " + output_word

            # Stop if we predict end of sentence or reach max words
            if output_word in ["minutes.", "minutes", "bake", "serve"] or len(generated_text.split()) >= num_words:
                if not generated_text.endswith("."):
                    generated_text += "."
                break

        return generated_text.capitalize()

def main():
    print("Creating pizza recipe generator...")
    generator = SimplePizzaGenerator()

    # Prepare data
    print("Preparing data...")
    df = generator.create_data()
    X, y, max_seq_len = generator.prepare_data(df)
    generator.max_len = max_seq_len  # Update max_len based on actual data

    # Train model
    print("Training model...")
    generator.train(X, y, max_seq_len, epochs=25)

    # Generate recipes
    print("\nGenerated Recipes:")
    seed_texts = [
        "make pizza dough by mixing",
        "to prepare the dough combine",
        "start by mixing flour and",
        "for a delicious pizza first"
    ]
    for i, seed in enumerate(seed_texts):
        recipe = generator.generate_recipe(
            start_text=seed,
            num_words=60
        )
        print(f"\nRecipe {i+1}:")
        print(recipe)
        print("-" * 50)

if __name__ == "__main__":
    main()

Creating pizza recipe generator...
Preparing data...
Vocabulary size: 69
Training model...
Epoch 1/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 10s 111ms/step - accuracy: 0.0768 - loss: 4.0386 - val_accuracy: 0.1068 - val_loss: 3.4749
Epoch 2/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - accuracy: 0.1714 - loss: 3.2680 - val_accuracy: 0.2534 - val_loss: 2.8471
Epoch 3/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 74ms/step - accuracy: 0.2458 - loss: 2.7400 - val_accuracy: 0.3466 - val_loss: 2.4626
Epoch 4/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 5s 78ms/step - accuracy: 0.3341 - loss: 2.3619 - val_accuracy: 0.4161 - val_loss: 2.1703
Epoch 5/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 5s 76ms/step - accuracy: 0.4187 - loss: 2.0423 - val_accuracy: 0.4472 - val_loss: 1.9011
Epoch 6/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 5s 107ms/step - accuracy: 0.4447 - loss: 1.7866 - val_accuracy: 0.4559 - val_loss: 1.6631
Epoch 7/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 78ms/step - accuracy: 0.5008 - loss: 1.5810 - val_accuracy: 0.5230 - val_loss: 1.4673
Epoch 8/25
51/51 